In [18]:
import numpy as np

#Bementi adatok
Aram = np.array([0.283,0.371,0.416]) #[A]
Fordulat = np.array([3718,4633,5534]) #[1/min]
Uk = np.array([6.5 , 8 , 9.5]) #[V]
#Elrendezésben megadott adatok
Tau = 0.325#[ms]  A tranziens folyamat Tau időállandója (alsó görbe)
Generator_ellenallas = 10
Uk_elrendezes = 8.5
Tau_kapocs = 0.34 #[s] #A kapocsfeszültség felfutásának időállandója (felső görbe)

def LMS (I , n , Uk):
    w = (n/60) * 2*3.1415 # Percenkénti fordulat átváltása szögsebességre (1/sec)
    #Mátrix definiálása
    Matrix = [[np.sum(I**2) , np.sum(w*I) , np.sum(Uk*I)],
              [np.sum(I*w) , np.sum(w**2) , np.sum(Uk*w)]]
    #Gauss elimináció
    x = Matrix[1][0] / Matrix[0][0]
    Matrix[1][0] = Matrix[1][0] - Matrix[0][0]*x
    Matrix[1][1] = Matrix[1][1] - Matrix[0][1]*x
    Matrix[1][2] = Matrix[1][2] - Matrix[0][2]*x
    #k és R kiszámítása
    k = (Matrix[1][2]/ Matrix[1][1])
    R = (Matrix[0][2] - k*Matrix[0][1])/Matrix[0][0]
    k = k*1000   #Azért kell a szorzás hogy mili legyen a mértékegység
    return (k,R)

def Viszkoz(k, I , n):
    w = (n/60) * 2*3.1415
    b = (k*np.sum(I*w)) / np.sum(w**2) #Lináris regresszió LMS módszerrel
    return (b)

def calc_Induktivitas(Tau, R_tekercs, R_generator):
    R_eredo = R_tekercs+R_generator #Induktivitásnál a generátor és a tekercs ellenállás eredőjável kell számolni
    L = Tau * R_eredo #Tau = L/R
    return (L)

def calc_Teta (Tau , k , R_tekercs , R_generator , b):
    k = k/1000 #k értékének ezred részével kell számolni (vagyis Nm/A)
    b = b/1000 #b értékének ezred részével kell számolni (vagyis Nms/rad)
    R = R_tekercs + R_generator #Tetához a tekercs és a generátor eredője kell
    Teta = (((k**2)/R) + b) * Tau #Felhasználva Tau = Teta / (((k**2)/R) + b) összefüggést
    return (Teta * (10**7))


def calc_A_B_T (k, R_tekercs, b , Teta):  # csak a tekercs ellenállása kell valamint k-t és b-t nem milliben kell megadni
    k = k/1000
    b = b/1000
    R = R_tekercs
    i = ((k**2)/R) + b
    
    T = (Teta / i) / 10000
    A = (k/R) / i 
    B = 1 / i
    
    return (A, B , T)

Motor = LMS(Aram,Fordulat,Uk)
b = Viszkoz(Motor[0], Aram , Fordulat)
L = calc_Induktivitas(Tau,Motor[1],Generator_ellenallas)
Teta = calc_Teta(Tau_kapocs , Motor[0] , Motor[1] , Generator_ellenallas , b)
A_B_T = calc_A_B_T(Motor[0] , Motor[1] , b , Teta)


print("A motor ellenállása:\t" , Motor[1] , "[Ohm]")
print("A motor k paramétere:\t" , Motor[0], "[mNm/A]")
print("A motor b paramétere:\t" , b, "[mNms/rad]")
print("A motor induktivitása:\t" , L, "[mH]")
print("A motor Teta értéke:\t" , Teta, "[gcm^2]")
print("A motor A paramétere:\t" , A_B_T[0], "[rad/Vs]")
print("A motor B paramétere:\t" , A_B_T[1], "[rad/sNm]")
print("A motor T paramétere:\t" , A_B_T[2], "[ms]")

A motor ellenállása:	 0.952478389923199 [Ohm]
A motor k paramétere:	 15.787931106264013 [mNm/A]
A motor b paramétere:	 0.011604228192282186 [mNms/rad]
A motor induktivitása:	 3.55955547672504 [mH]
A motor Teta értéke:	 116.83227911213004 [gcm^2]
A motor A paramétere:	 60.65013787257038 [rad/Vs]
A motor B paramétere:	 3658.994030355623 [rad/sNm]
A motor T paramétere:	 42.74886118241258 [ms]
